# TCP, DNS, HTTP/2 vs HTTP/3 Lab
Objectives:
- TCP handshake & congestion control notes
- DNS resolution walkthrough (socket/getaddrinfo, dig output paste)
- HTTP/1.1 vs HTTP/2 vs HTTP/3 discussion (pcap placeholder)
- Small HTTP client with timeouts

In [ ]:
# Starter imports
import socket
import ssl
from pathlib import Path
import json
import time

# TODO: add experiments for TCP handshake, DNS queries, and HTTP/2 vs HTTP/3

In [ ]:
import socket
from contextlib import closing
from typing import Tuple

def tcp_handshake(host: str, port: int, timeout: float = 3.0) -> Tuple[str, str]:
    """Open a TCP socket to show local/remote addresses; closes immediately."""
    with closing(socket.create_connection((host, port), timeout=timeout)) as s:
        local = f"{s.getsockname()[0]}:{s.getsockname()[1]}"
        remote = f"{s.getpeername()[0]}:{s.getpeername()[1]}"
    return local, remote

try:
    print(tcp_handshake("example.com", 80))
except OSError as exc:
    print(f"TCP connect failed: {exc}")

In [ ]:
import ipaddress

def dns_lookup(host: str):
    infos = socket.getaddrinfo(host, None)
    uniq = {}
    for family, _, _, _, sockaddr in infos:
        ip = sockaddr[0]
        uniq[ip] = ipaddress.ip_address(ip).version
    return uniq

dns_lookup("example.com")

### HTTP/1.1 vs HTTP/2 vs HTTP/3 notes
- HTTP/1.1: plain TCP + optional TLS; head-of-line blocking at TCP layer.
- HTTP/2: multiplexed streams over a single TCP connection; HPACK compression; still HOL at TCP.
- HTTP/3: QUIC over UDP with streams + TLS 1.3 built in; avoids TCP HOL; connection migration.
- Pcaps: capture with `tcpdump -i any "port 80 or port 443" -w http.pcap` then inspect stream counts.
- Tooling: `curl -v --http2` and `curl -v --http3` to compare negotiations.

In [ ]:
import http.client

def http_get(host: str, path: str = "/", use_tls: bool = False, timeout: float = 3.0):
    conn_cls = http.client.HTTPSConnection if use_tls else http.client.HTTPConnection
    conn = conn_cls(host, timeout=timeout)
    try:
        conn.request("GET", path)
        resp = conn.getresponse()
        body = resp.read(200)  # preview only
        return {
            "status": resp.status,
            "reason": resp.reason,
            "headers": dict(resp.getheaders()),
            "body_preview": body.decode("utf-8", errors="ignore"),
        }
    finally:
        conn.close()

try:
    http_get("example.com", "/")
except Exception as exc:
    print(f"HTTP request failed: {exc}")

In [ ]:
import time

def with_retries(op, attempts=3, backoff=0.5):
    for i in range(attempts):
        try:
            return op()
        except Exception as exc:
            if i == attempts - 1:
                raise
            sleep = backoff * (2 ** i)
            print(f"retry {i+1}/{attempts} after {exc}, sleeping {sleep:.2f}s")
            time.sleep(sleep)

# Example usage (commented to avoid repeated external calls)
# with_retries(lambda: http_get("example.com", "/"))